In [36]:
import tensorflow as tf
import numpy as np

data_x = [[-1.,-2.,1.],[-2.,-3.,1.],[-3,-4.,1.],[1.,2.,1.],[2.,3.,1,],[3.,4.,1.]]
data_y = [[1.,0.],[1.,0.],[1.,0.],[0.,1.],[0.,1.],[0.,1.]]

batch_size = len(data_x);
x = tf.placeholder(shape=[batch_size,3], dtype=tf.float32, name="x")
y = tf.placeholder(shape=[batch_size,2], dtype=tf.float32, name="y")

wrt_variables = tf.Variable(tf.ones(shape=[6,1]), dtype=tf.float32, name="wrt")
z = tf.Variable(tf.ones(shape=[6,1]), dtype=tf.float32, name="z")
wrt_variables_list = tf.unstack(wrt_variables)
pred = tf.nn.softmax(tf.matmul(x, tf.reshape(wrt_variables_list,[3,2])))

loss = -tf.reduce_sum(y*tf.log(pred),reduction_indices=[1])
loss = tf.reshape(loss,[6,1])
G = tf.gradients(loss,wrt_variables_list)
temp2 = grad
cg_wrt_variables = z

for iteration in range(1):
    cg_wrt_variables_list = tf.unstack(cg_wrt_variables)
    R_op = Rop(loss,wrt_variables,cg_wrt_variables)
    L_op = tf.transpose(Lop(loss,wrt_variables_list,R_op))
    cg_loss = (G-L_op)**2
    
    [H,G] = Hess_Comp(cg_loss,cg_wrt_variables_list);
    d=-G;
    for index in range (0,len(cg_wrt_variables_list)-1):
        Alpha_N= tf.matmul(-tf.transpose(G),d);
        Alpha_D= tf.matmul(tf.matmul(tf.transpose(d),H),d);
        Alpha = tf.divide(Alpha_N,Alpha_D);
        cg_update_directions = Alpha*d    

        cg_wrt_variables = tf.assign_add(cg_wrt_variables,cg_update_directions)
        temp = Rop(loss,wrt_variables_list,cg_wrt_variables_list)

        cg_wrt_variables_list = tf.unstack(cg_wrt_variables);
        R_op = Rop(loss,wrt_variables,cg_wrt_variables)
        L_op = tf.transpose(Lop(loss,wrt_variables_list,R_op))
        cg_loss = (grad-L_op)**2

        [H1,G]=Hess_Comp(cg_loss,cg_wrt_variables_list)
        Beta_N= tf.matmul(tf.matmul(tf.transpose(G),H),d);
        Beta_D= tf.matmul(tf.matmul(tf.transpose(d),H),d);
        Beta = tf.divide(Beta_N,Beta_D);
        Beta_update_directions = tf.unstack(Beta*d)
        d=tf.subtract(Beta_update_directions,G)
        [H,G]=Hess_Comp(cg_loss,cg_wrt_variables_list)
update_directions = -cg_wrt_variables;
update_directions=(tf.unstack(update_directions))

wrt_variables = tf.assign_add(wrt_variables,update_directions)



In [1]:
def Jaco_comp(y,x_list):
    Jaco=[];Jaco1=[];
    Jaco.append([tf.gradients(y_, x_list) for y_ in y])
    for i in range(0,len(Jaco[0])):
        for j in range(0,len(Jaco[0][i])):
            if Jaco[0][i][j] == None:
                Jaco1.append(cons(0))   
            else: 
                Jaco1.append(Jaco[0][i][j])
    Jaco1 = tf.reshape(Jaco1,[len(y),len(x_list)])
    return(Jaco1)

In [2]:
def Hess_Comp(y,x_list):
    grads=[];H=[];grads1=[];H1=[];
    grads.append(tf.gradients(y, x_list))
    for i in range(0,len(grads[0])):
        if grads[0][i] == None:
            grads1.append(cons(10))   
        else: 
            grads1.append(grads[0][i])
    H.append([tf.gradients(y_, x_list) for y_ in grads1])
    for i in range(0,len(H[0])):
        for j in range(0,len(H[0][i])):
            if H[0][i][j] == None:
                H1.append(cons(10))   
            else: 
                H1.append(H[0][i][j])
    H1 = tf.reshape(H1,[len(x_list),len(x_list)])
    grads1 = tf.reshape(grads1,[len(x_list),1])

    return(H1,grads1)

In [21]:
def cons(x):
    return tf.constant(x, dtype=tf.float32)

In [37]:


sess = tf.Session()
print("Hi")

sess.run(tf.global_variables_initializer())

initial_loss = sess.run(
    loss,
    feed_dict={
        x: data_x,
        y: data_y
    }
)

for iteration in range(10):
    g1,g,_,q = sess.run(
        [wrt_variables,cg_loss,R_op,temp2],
        feed_dict={
            x: data_x,
            y: data_y
        }
    )
    print("Grad:",q)    

    comp_loss = sess.run(
        loss,
        feed_dict={
            x: data_x,
            y: data_y
        }
    )
    print("iteration {}: {}".format(iteration+1,sum(comp_loss)/len(data_x)))
    if(sum(comp_loss)/len(data_x)<0.001):
        break
        
print("Wrt_var:",g1)
print("Prediction:", sess.run(pred, feed_dict={x: data_x}))
print("Expected:", data_y)

sess.close();

Hi
Grad: [array([ 6.], dtype=float32), array([-6.], dtype=float32), array([ 9.], dtype=float32), array([-9.], dtype=float32), array([ 0.], dtype=float32), array([ 0.], dtype=float32)]
iteration 1: [ nan]
Grad: [array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32)]
iteration 2: [ nan]
Grad: [array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32)]
iteration 3: [ nan]
Grad: [array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32)]
iteration 4: [ nan]
Grad: [array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtype=float32), array([ nan], dtyp

C:\Users\suraj\AppData\Local\Continuum\anaconda2\envs\SurajKiranRaman\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in less


Prediction: [[ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]]
Expected: [[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]


In [61]:
def gradients(f, x, grad_ys=None):
    '''
    An easier way of computing gradients in tensorflow. The difference from tf.gradients is
        * If f is not connected with x in the graph, it will output 0s instead of Nones. This will be more meaningful
            for computing higher-order gradients.
        * The output will have the same shape and type as x. If x is a list, it will be a list. If x is a Tensor, it
            will be a tensor as well.
    :param f: A `Tensor` or a list of tensors to be differentiated
    :param x: A `Tensor` or a list of tensors to be used for differentiation
    :param grad_ys: Optional. It is a `Tensor` or a list of tensors having exactly the same shape and type as `f` and
                    holds gradients computed for each of `f`.
    :return: A `Tensor` or a list of tensors having the same shape and type as `x`
    '''

    if isinstance(x, list):
        grad = tf.gradients(f, x, grad_ys=grad_ys)
        for i in range(len(x)):
            if grad[i] is None:
                grad[i] = tf.zeros_like(x[i])
        return grad
    else:
        grad = tf.gradients(f, x, grad_ys=grad_ys)[0]
        if grad is None:
            return tf.zeros_like(x)
        else:
            return grad


def Lop(f, x, v):
    '''
    Compute Jacobian-vector product. The result is v^T @ J_x
    :param f: A `Tensor` or a list of tensors for computing the Jacobian J_x
    :param x: A `Tensor` or a list of tensors with respect to which the Jacobian is computed.
    :param v: A `Tensor` or a list of tensors having the same shape and type as `f`
    :return: A `Tensor` or a list of tensors having the same shape and type as `x`
    '''
    assert not isinstance(f, list) or isinstance(v, list), "f and v should be of the same type"
    return gradients(f, x, grad_ys=v)


def Rop(f, x, v):
    '''
    Compute Jacobian-vector product. The result is J_x @ v.
    The method is inspired by [deep yearning's blog](https://j-towns.github.io/2017/06/12/A-new-trick.html)
    :param f: A `Tensor` or a list of tensors for computing the Jacobian J_x
    :param x: A `Tensor` or a list of tensors with respect to which the Jacobian is computed
    :param v: A `Tensor` or a list of tensors having the same shape and type as `v`
    :return: A `Tensor` or a list of tensors having the same shape and type as `f`
    '''
    assert not isinstance(x, list) or isinstance(v, list), "x and v should be of the same type"
    if isinstance(f, list):
        w = [tf.ones_like(_) for _ in f]
    else:
        w = tf.ones_like(f)
    return (gradients(Lop(f, x, w), w, grad_ys=v))

In [66]:
data_w =[[1, 1], [1, 1]]
data_v = [[2, 2], [2, 2]]
data_x = [[0,1]]

In [64]:
data_v = [[2, 2]]
data_x = [[0,1]]

In [67]:
x = tf.placeholder(shape=[1,2], dtype=tf.float32, name="x")
V = tf.placeholder(shape=[2,2], dtype=tf.float32, name="V")
W = tf.placeholder(shape=[2,2], dtype=tf.float32, name="W")

In [68]:
y = tf.matmul(x,W)

In [70]:
ROP_val = Rop(y, W, V)

In [71]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Rop_var = sess.run(
    ROP_val,
    feed_dict={
        W: data_w,
        V: data_v,
        x: data_x
    }
)

In [72]:
print(Rop_var)

[[ 2.  2.]]
